This notebook shows the whole process of how I extract feature, build the model, make prediction and calculate rank sum score. (Might be messy)

All the functions here are written seperately and nicely in /lib file.

In [1]:
import os
import collections
import hdf5_getters
import tables
import numpy as np
from sklearn import mixture
import pandas as pd
from scipy.stats import rankdata

In [2]:
def get_file_path(startdir):
    path = []
    file_name = []
    for dirs, subdirs, files in os.walk(startdir):
        for name in files:
            if name != ".DS_Store":
                file_name.append(name)
                path_s = os.path.join(dirs, name)
                path.append(path_s)
    return(path)

In [3]:
def get_file_name(startdir):
    file_name = []
    for dirs, subdirs, files in os.walk(startdir):
        for name in files:
            if name != ".DS_Store":
                file_name.append(name)
    return(file_name)

In [4]:
def get_list_attr(path_list, attr):
    attr_list = []
    i = 1
    for file in path_list:
        try:
            file_read = hdf5_getters.open_h5_file_read(file)
            #print(hdf5_getters.__getattribute__(attr)(file_read))
            attr_list.append(hdf5_getters.__getattribute__(attr)(file_read))
            
            file_read.close()
            #print 'Finished ' + str(i) + '/2350'
            i += 1
        except:
            print '---- Failed to get ' + file + ' ---- No:' + str(i)
            
            attr_list.append(0)
            i += 1
    return(attr_list)

In [5]:
def stat(attr):
    ls = []
    for l in attr:
        try:
            stats = [np.max(l), np.min(l), np.mean(l), np.var(l), np.ptp(l), np.percentile(l, 25), np.percentile(l, 50), np.percentile(l, 75)]
            ls.append(stats)
        except ValueError:
            ls.append([0] * 8)
    return(ls)

In [6]:
def cmbfat(attr_ls):
    ls0 = attr_ls[0]
    for l in attr_ls[1:]:
        for i in range(len(ls0)):
            ls0[i] += l[i]
    return(ls0)

In [56]:
def em_model(df_train, num_cluster):
    
    # return EM model
    
    df = pd.DataFrame(df_train,  index = df_train.index.tolist())
    gmm = mixture.GaussianMixture(n_components = num_cluster, covariance_type='full', random_state = 0)
    #em = mixture.GMM(n_components = num_cluster, random_state = 0)
    #model = em.fit(df_train)
    model = gmm.fit(df_train)
    return(model)

def word_cluster_table(model, df_train):
    
    # return word frequency for each cluster

    pred = model.predict(df_train)
    df_cluster = pd.DataFrame({'file_1': df_train.index.tolist(), 'cluster':pred, 'count': 1})
    df_word = pd.read_csv('lyr.csv')
    
    for i in range(len(pd.Series.unique(df_cluster['cluster']))):
        if i == 0:
            tmp_ini = df_cluster.loc[df_cluster['cluster'] == i]
            result_ini = pd.concat([tmp_ini, df_word], axis=1, join='inner')
            df_s_ini = result_ini.drop('file_1', 1)
            #cnt_ini = len(df_s_ini)
            sum_c_ini = df_s_ini.sum(axis=0)
            ls_wd = sum_c_ini.to_frame()
            #ls_word['cnt'] = cnt_ini
        else:
            tmp = df_cluster.loc[df_cluster['cluster'] == i]
            result = pd.concat([tmp, df_word], axis=1, join='inner')
            df_s = result.drop('file_1', 1)
            cnt = len(df_s)
            sum_c = df_s.sum(axis=0)
            ls_sum = list(sum_c)
            #ls_sum.append
            ls_wd[i] = ls_sum
        ls_word = ls_wd.transpose()
        ls_word = ls_word.drop('dat2$track_id', 1).drop('cluster', 1)
        
    return(ls_word)

In [57]:
def test_model(cluster_table, model, df_test):
    
    # return predicted word rank
    # pred_test = model.predict(df_test)
    cluster_table['cluster'] = range(len(cluster_table))
    pred_test = model.predict(df_test)
    pred_cluster = pd.DataFrame({'file_1': df_test.index.tolist(), 'cluster':pred_test})

    for i in range(len(pred_cluster)):
        if i == 0:
            pre_c_word = cluster_table.loc[cluster_table['cluster'] == pred_cluster.ix[i][0]]
            pre_c_word['track_id'] = pred_cluster.ix[i][1]
        else:
            tmp = cluster_table.loc[cluster_table['cluster'] == pred_cluster.ix[i][0]]
            tmp['track_id'] = pred_cluster.ix[i][1]
            pre_c_word = pd.concat([pre_c_word, tmp])
    index = pre_c_word['track_id'].tolist()
    word_test = pre_c_word.ix[:,'blank_': 'zwei']
    word_test.index = index
    
    
    for i in range(len(word_test)):
        ls_freq = word_test.iloc[[i]].values.tolist()[0]
        ls_ne = [-x for x in ls_freq]
        rank = rankdata(ls_ne).tolist()
        #seq = sorted(ls_freq, reverse = True)
        #rank = [seq.index(v)+1 for v in ls_freq]
        word_test.iloc[i] = rank
    
    return(word_test)


In [48]:
def eval_score(test_words, rank_table):
    
    test_w = test_words.ix[:,'blank_':]
    test_w.index = test_words['dat2$track_id'].tolist()
    
    results = []
    for i in range(len(test_w.index)):
        ls = test_w.ix[i].nonzero()[0].tolist()
        words_rank = rank_table.ix[i].iloc[ls].tolist()
        rank_bar = sum(words_rank)/(len(ls) * sum(rank_table.ix[i])/5000)
        results.append(rank_bar)
    return(results)
    

In [10]:
startdir = '/Users/dd/Desktop/Columbia/Applied Data Science/Project4/TestSongFile100'
path = get_file_path(startdir)

In [13]:
segments_start = get_list_attr(path, 'get_segments_start')
print(1)
segments_confidence = get_list_attr(path, 'get_segments_confidence')
print(1)
segments_pitches = get_list_attr(path, 'get_segments_pitches')
print(1)
segments_timbre = get_list_attr(path, 'get_segments_timbre')
print(1)
segments_loudness_max = get_list_attr(path, 'get_segments_loudness_max')
print(1)
segments_loudness_max_time = get_list_attr(path, 'get_segments_loudness_max_time')
print(1)
segments_loudness_start = get_list_attr(path, 'get_segments_loudness_start')
print(1)
sections_start = get_list_attr(path, 'get_sections_start')
print(1)
sections_confidence = get_list_attr(path, 'get_sections_confidence')
print(1)
beats_start = get_list_attr(path, 'get_beats_start')
print(1)
beats_confidence = get_list_attr(path, 'get_beats_confidence')
print(1)
bars_start = get_list_attr(path, 'get_bars_start')
print(1)
bars_confidence = get_list_attr(path, 'get_bars_confidence')
print(1)
tatums_start = get_list_attr(path, 'get_tatums_start')
print(1)
tatums_confidence = get_list_attr(path, 'get_tatums_confidence')
print(1)

ls_feature = [segments_start, segments_confidence, segments_pitches, segments_timbre, segments_loudness_max, segments_loudness_max_time,
             segments_loudness_start, sections_start, sections_confidence, beats_start, beats_confidence, bars_start,
             bars_confidence, tatums_start, tatums_confidence]



1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [14]:
segments_start_s = stat(segments_start)
segments_confidence_s = stat(segments_confidence)
segments_pitches_s = stat(segments_pitches)
segments_timbre_s = stat(segments_timbre)
segments_loudness_max_s = stat(segments_loudness_max)
segments_loudness_max_time_s = stat(segments_loudness_max_time)
segments_loudness_start_s = stat(segments_loudness_start)
sections_start_s = stat(sections_start)
sections_confidence_s = stat(sections_confidence)
beats_start_s = stat(beats_start)
beats_confidence_s = stat(beats_confidence)
bars_start_s = stat(bars_start)
bars_confidence_s = stat(bars_confidence)
tatums_start_s = stat(tatums_start)
tatums_confidence_s = stat(tatums_confidence)

ls_feature_s = [segments_start_s, segments_confidence_s, segments_pitches_s, segments_timbre_s, segments_loudness_max_s, segments_loudness_max_time_s,
             segments_loudness_start_s, sections_start_s, sections_confidence_s, beats_start_s, beats_confidence_s, bars_start_s,
             bars_confidence_s, tatums_start_s, tatums_confidence_s]

In [15]:
feature = cmbfat(ls_feature_s)

file_name = get_file_name(startdir)
file_n = []
for f in file_name:
    file_n.append(f[:-3])

headers = file_n

df = pd.DataFrame(feature, index = headers)
df.to_pickle('/Users/dd/Desktop/Columbia/Applied Data Science/Project4/feature_real.pickle')

In [58]:
df_train = pd.read_pickle('/Users/dd/Desktop/Columbia/Applied Data Science/Project4/feature_train.pickle')

In [59]:
em_model = em_model(df_train, 10)

In [60]:
word_cluster_table = word_cluster_table(em_model, df_train)


In [61]:
rank_table = test_model(word_cluster_table, em_model, df)

/Users/dd/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dd/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [111]:
rank_table

,blank_,–,¿qué,‘caus,&,000,1,10,100,12,...,zijn,zo,zombi,zone,zoo,zu,zum,zur,zurück,zwei
1,975.5,1369,4585.5,3247,774,2774.5,1128,1588,2398.5,4585.5,...,4585.5,4585.5,2398.5,2125,1735,723.5,1909,3835.5,2774.5,2774.5
10,975.5,1369,4585.5,3247,774,2774.5,1128,1588,2398.5,4585.5,...,4585.5,4585.5,2398.5,2125,1735,723.5,1909,3835.5,2774.5,2774.5
100,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
11,975.5,1369,4585.5,3247,774,2774.5,1128,1588,2398.5,4585.5,...,4585.5,4585.5,2398.5,2125,1735,723.5,1909,3835.5,2774.5,2774.5
12,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
13,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
14,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
15,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
16,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5
17,4821.5,4078,4447,4447,473.5,4447,513,2662.5,2204.5,2474.5,...,4821.5,4821.5,4078,1483,4447,1483,3739.5,4078,3122,4821.5


In [72]:
ix_name = rank_table.index.values.tolist()

In [99]:
ix_n2 = [int(ix[8:]) for ix in ix_name]

In [107]:
rank_table_s = rank_table

In [108]:
rank_table_s.index = ix_n2

In [112]:
success = rank_table_s.sort_index()

In [114]:
success.to_csv('final_result.csv')

In [ ]:
# calculate predictive rank sum on test data

'''
file_name = get_file_name('/Users/dd/Desktop/Columbia/Applied Data Science/Project4/data')
file_n = []
for f in file_name:
    file_n.append(f[:-3])

headers = file_n

df = pd.DataFrame(feature, index = headers)

# select 2200 for training, 150 for testing
df_train = df[:2200]
df_test = df[2200:]
'''
train_ = df_train[:2000]
test_ = df_train[2000:]

em_model_2 = em_model(train_, 8)
word_cluster_table_2 = word_cluster_table(em_model_2, train_)
rank_table_2 = test_model(word_cluster_table_2, em_model_2, test_)
test_words = pd.read_csv('lyr.csv')
test_words = test_words[2000:]
score = eval_score(test_words, rank_table_2)
np.mean(score)

In [35]:
# baseline Method Rank Sum

test_words
test_w = test_words.ix[:,'blank_':]
test_w.index = test_words['dat2$track_id'].tolist()
f = test_w.sum(axis = 0)
test = rank_table

for i in range(len(test)):
    ls_ne = [-x for x in f]
    rank = rankdata(ls_ne).tolist()
    test.iloc[i] = rank
score_base = eval_score(test_words, test)

In [37]:
np.mean(score_base)

0.20229251659321643